### Edible Mushroom Classification using Decision trees

#### Problem Statement - 

I have a dataset of mushrooms with three attributes. I classify the data to tell whether a given mushroom is edible or poisonous based on these physical attributes.


#### Dataset - 

The dataset we have collected is as follows:

| Cap Color | Stalk Shape | Solitary | Edible |
|:---------:|:-----------:|:--------:|:------:|
|   Brown   |   Tapering  |    Yes   |    1   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    No    |    0   |
|   Brown   |   Tapering  |    Yes   |    1   |
|    Red    |   Tapering  |    Yes   |    0   |
|    Red    |  Enlarging  |    No    |    0   |
|   Brown   |  Enlarging  |    Yes   |    1   |
|    Red    |   Tapering  |    No    |    1   |
|   Brown   |  Enlarging  |    No    |    0   |


There are 10 examples of mushrooms. For each example, there are
- Three features
    - Cap Color (`Brown` or `Red`),
    - Stalk Shape (`Tapering` or `Enlarging`),
    - Solitary (`Yes` or `No`)
- Label
    - Edible (`1` indicating yes or `0` indicating poisonous)


#### One hot encoded dataset -

| Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:---------:|:--------------------:|:--------:|:------:|
|     1     |           1          |     1    |    1   |
|     1     |           0          |     1    |    1   |
|     1     |           0          |     0    |    0   |
|     1     |           0          |     0    |    0   |
|     1     |           1          |     1    |    1   |
|     0     |           1          |     1    |    0   |
|     0     |           0          |     0    |    0   |
|     1     |           0          |     1    |    1   |
|     0     |           1          |     0    |    1   |
|     1     |           0          |     0    |    0   |

- `X_train` contains three features for each example 
    - Brown Color (`1` - "Brown" and `0` - "Red")
    - Tapering Shape (`1` - "Tapering Stalk Shape" and `0` - "Enlarging stalk shape")
    - Solitary  (`1` - "Yes" and `0` - "No")

- `y_train` is whether the mushroom is edible 
    - `y = 1` - edible
    - `y = 0` - poisonous

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
X_train = np.array([[1,1,1],[1,0,1],[1,0,0],[1,0,0],[1,1,1],[0,1,1],[0,0,0],[1,0,1],[0,1,0],[1,0,0]])
y_train = np.array([1,1,0,0,1,0,0,1,1,0])

In [4]:
print("First few elements of X_train:\n", X_train[:5])

First few elements of X_train:
 [[1 1 1]
 [1 0 1]
 [1 0 0]
 [1 0 0]
 [1 1 1]]


In [5]:
print("First few elements of y_train:", y_train[:5])


First few elements of y_train: [1 1 0 0 1]


In [6]:
print ('The shape of X_train is:', X_train.shape)
print ('The shape of y_train is: ', y_train.shape)
print ('Number of training examples (m):', len(X_train))

The shape of X_train is: (10, 3)
The shape of y_train is:  (10,)
Number of training examples (m): 10


####  Decision Tree

- The steps for building a decision tree are as follows:
    - Start with all examples at the root node
    - Calculate information gain for splitting on all possible features, and pick the one with the highest information gain
    - Split dataset according to the selected feature, and create left and right branches of the tree
    - Keep repeating splitting process until stopping criteria is met
  
  
- In this project I did the following -
    - Calculate the entropy at a node 
    - Split the dataset at a node into left and right branches based on a given feature
    - Calculate the information gain from splitting on a given feature
    - Choose the feature that maximizes information gain
    
- The stopping criteria for this project is set at a maximum depth of 2

#### Calculating entropy

First, I've created a helper function called `compute_entropy` that computes the entropy (measure of impurity) at a node. 
- The function takes in a numpy array (`y`) that indicates whether the examples in that node are edible (`1`) or poisonous(`0`) 

$$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$
* Note 
    
    * For implementation purposes, $0\text{log}_2(0) = 0$. That is, if `p_1 = 0` or `p_1 = 1`, set the entropy to `0`

In [7]:
def compute_entropy(y):
    
    entropy = 0.
    
    if len(y) != 0 :
        p1 = len(y[y==1])/len(y)

        if p1 != 1 and p1 != 0 :
            entropy = -p1*(np.log2(p1)) -((1-p1)*np.log2(1-p1))
        else:
            entropy = 0
       
    
    return entropy

####  Split dataset

Next, I've created another helper function called `split_dataset` that takes in the data at a node and a feature to split on and splits it into left and right branches.

- The function takes in the training data, the list of indices of data points at that node, along with the feature to split on. 
- It splits the data and returns the subset of indices at the left and the right branch.
- For example, say I'm starting at the root node (so `node_indices = [0,1,2,3,4,5,6,7,8,9]`), and I choose to split on feature `0`, which is whether or not the example has a brown cap. 
    - The output of the function is then, `left_indices = [0,1,2,3,4,7,9]` (data points with brown cap) and `right_indices = [5,6,8]` (data points without a brown cap)
    
    
| Brown Cap | Tapering Stalk Shape | Solitary | Edible |
|:---------:|:--------------------:|:--------:|:------:|
|     1     |           1          |     1    |    1   |
|     1     |           0          |     1    |    1   |
|     1     |           0          |     0    |    0   |
|     1     |           0          |     0    |    0   |
|     1     |           1          |     1    |    1   |
|     0     |           1          |     1    |    0   |
|     0     |           0          |     0    |    0   |
|     1     |           0          |     1    |    1   |
|     0     |           1          |     0    |    1   |
|     1     |           0          |     0    |    0   |
    

In [8]:
def split_dataset(X, node_indices, feature):
    
    left_indices = []
    right_indices = []
    
    for i in node_indices :
        if X[i][feature] == 1 :
            left_indices.append(i)
        else:
            right_indices.append(i)
        
    return left_indices, right_indices

Now, let's try splitting the dataset at the root node, which contains all examples at feature 0 (Brown Cap).

In [9]:
# Case 1

root_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
feature = 0
left_indices, right_indices = split_dataset(X_train, root_indices, feature)

print("CASE 1:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)


# Case 2

root_indices_subset = [0, 2, 4, 6, 8]
left_indices, right_indices = split_dataset(X_train, root_indices_subset, feature)

print("CASE 2:")
print("Left indices: ", left_indices)
print("Right indices: ", right_indices)



CASE 1:
Left indices:  [0, 1, 2, 3, 4, 7, 9]
Right indices:  [5, 6, 8]
CASE 2:
Left indices:  [0, 2, 4]
Right indices:  [6, 8]


#### Calculate information gain

Next, I've written a function called `information_gain` that takes in the training data, the indices at a node and a feature to split on and returns the information gain from the split.

In [10]:
def compute_information_gain(X, y, node_indices, feature):
      
    # Split dataset
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]
    
    information_gain = 0
    
    node_entropy = compute_entropy(y_node)
    left_entropy = compute_entropy(y_left)
    right_entropy = compute_entropy(y_right)
    
    w_left = len(X_left)/len(X_node)
    w_right = len(X_right)/len(X_node)
    
    weighed_entropy = w_left*left_entropy + w_right*right_entropy
    information_gain = node_entropy - weighed_entropy
    
    return information_gain

In [11]:
info_gain0 = compute_information_gain(X_train, y_train, root_indices, feature=0)
print("Information Gain from splitting the root on brown cap: ", info_gain0)

info_gain1 = compute_information_gain(X_train, y_train, root_indices, feature=1)
print("Information Gain from splitting the root on tapering stalk shape: ", info_gain1)

info_gain2 = compute_information_gain(X_train, y_train, root_indices, feature=2)
print("Information Gain from splitting the root on solitary: ", info_gain2)


Information Gain from splitting the root on brown cap:  0.034851554559677034
Information Gain from splitting the root on tapering stalk shape:  0.12451124978365313
Information Gain from splitting the root on solitary:  0.2780719051126377


Splitting on "Solitary" (feature = 2) at the root node gives the maximum information gain. Therefore, it's the best feature to split on at the root node.

#### Get best split
Next I've written a function to get the best feature to split on by computing the information gain from each feature as I did above and returning the feature that gives the maximum information gain

In [12]:
def get_best_split(X, y, node_indices):   
    
    num_features = X.shape[1]
    best_feature = -1
    
    max_info_gain = 0
    
    for feature in range(num_features) :
        info_gain = compute_information_gain(X, y, node_indices, feature)
        
        if info_gain > max_info_gain :
            max_info_gain = info_gain
            best_feature = feature
   
    return best_feature

In [13]:
best_feature = get_best_split(X_train, y_train, root_indices)
print("Best feature to split on: %d" % best_feature)


Best feature to split on: 2


As it was above above, the function returns that the best feature to split on at the root node is feature 2 ("Solitary")

#### Building the tree

In this section a decision tree is generated by successively picking the best feature to split on until the stopping criteria is reached (maximum depth is 2).

In [14]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    
    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return
   
    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices) 
    
    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))
    
    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))
    
    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)

In [15]:
build_tree_recursive(X_train, y_train, root_indices, "Root", max_depth=2, current_depth=0)


 Depth 0, Root: Split on feature: 2
- Depth 1, Left: Split on feature: 0
  -- Left leaf node with indices [0, 1, 4, 7]
  -- Right leaf node with indices [5]
- Depth 1, Right: Split on feature: 1
  -- Left leaf node with indices [8]
  -- Right leaf node with indices [2, 3, 6, 9]
